<img src='https://i.imgur.com/RDAD11M.png' width = '200' align = 'right'>

## *DATA SCIENCE / SECTION 1 / SPRINT 2 / NOTE 1*

# 📝 Assignment

# 가설 검정

## 1. One-Sample T-test 연습

### 데이터 불러오기
- [서울시 가로수 데이터셋](https://data.seoul.go.kr/dataList/367/S/2/datasetView.do)을 사용합니다.

다음 링크를 참조하세요. 
<https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/seoul_tree/seoul_tree.txt>


In [ ]:
import pandas as pd
import numpy as np
from scipy import stats

pd.set_option('mode.chained_assignment',  None)

In [ ]:
df = pd.read_csv('https://ds-lecture-data.s3.ap-northeast-2.amazonaws.com/seoul_tree/seoul_tree.txt', sep = "\t")

df.drop(df.index[(df['자치구'] == '서울시') | (df['자치구'] == '공원녹지사업소') | (df['자치구'] == '시설관리공단')], inplace = True)
df.columns = df.iloc[0] # 첫 행을 헤더로 지정
df = df[1:] # 헤더 지정한 행 제외 나머지 행 가져오기
df.reset_index(drop=True, inplace=True) # 인덱스 초기화

# '기간', '자치구'와 그 외 숫자 형변환할 컬럼을 분리
df_object = df.iloc[:, :2]
df_numeric = df.iloc[:, 2:]

# '기간', '자치구' 제외 나머지 feature에 대해서 수치형 데이터로 수정
df_numeric.replace({',': '', '-': '0'}, regex=True, inplace=True)
df_numeric = df_numeric.apply(pd.to_numeric)

# 분리했던 데이터프레임 합치기
df = pd.concat([df_object, df_numeric], axis=1)

df

# 열 나누기 참고 : [~df['자치구'].isin(['서울시','공원녹지사업소','시설관리공단'])

,기간,자치구,노선수,수량,은행나무,양버즘나무,느티나무,왕벚나무,은단풍나무,메타세콰이어,회화나무,가죽나무,감나무,수양버들,튜울립나무,칠엽수,포플러,느릅나무,살구나무,무궁화,소나무,목련,이팝나무,기타
0,2019,종로구,70,7329,4072,1071,619,421,0,1,140,1,0,8,0,17,0,0,0,0,239,0,711,29
1,2019,중구,48,8250,3640,162,460,289,0,0,176,0,0,0,7,0,0,0,21,46,2162,0,470,817
2,2019,용산구,39,7835,4444,1422,373,262,0,0,37,0,74,0,13,16,0,0,39,0,51,126,198,780
3,2019,성동구,38,8120,1090,2401,1975,602,29,48,325,33,146,0,0,1,3,59,0,0,53,0,709,646
4,2019,광진구,28,7223,1247,2306,1057,940,0,0,153,0,25,0,58,101,0,0,0,0,20,113,760,443
5,2019,동대문구,43,10664,3749,3342,855,747,0,60,143,7,38,7,37,19,0,1,26,1,0,0,1176,456
6,2019,중랑구,57,8365,2774,2759,471,354,0,74,0,0,52,0,0,0,0,0,0,212,64,0,449,1156
7,2019,성북구,78,8449,2722,1282,1014,1656,66,65,38,0,91,0,4,0,0,0,9,5,97,0,853,547
8,2019,강북구,31,7789,3510,934,324,497,284,0,103,0,0,0,5,0,0,140,0,0,1007,0,575,410
9,2019,도봉구,63,9572,4914,323,779,2095,6,61,32,0,0,1,15,0,0,0,0,214,15,0,47,1070


### "서울시에는 구별로 평균 약 400 그루 정도의 `이팝나무`가 있다"라는 가설에 대해 가설검정을 시행하세요.

- 단, `서울시`, `공원녹지사업소`, `시설관리공단` 부분의 데이터는 제외합니다.
- 가설검정 결과의 pvalue 값을 `pv1`에 저장하세요. (`.pvalue` 사용)

In [ ]:
# H0 = 서울시에는 구별로 평균 약 400 그루 정도의 이팝나무가 있다
# H1 = 서울시에는 구별로 평균 약 400 그루 정도의 이팝나무가 있지 않다(평균이 400이 아님)
e_pub = df['이팝나무']
print(np.mean(e_pub))

print(stats.ttest_1samp(e_pub, 400))

pv1 = stats.ttest_1samp(e_pub, 400).pvalue
pv1
# pv1 < 0.05 이므로 H0는 기각

688.64
Ttest_1sampResult(statistic=3.3809205562101288, pvalue=0.002470712850935819)


0.002470712850935819

### 이번에는 10개의 구를 샘플링하여, 위에 대한 가설검정을 다시 한번 시행해보세요.

- 단, 아래의 코드를 참조하여 sample을 위한 seed 는 123로 설정하세요.
- 가설검정 결과의 pvalue값을 `pv2`에 저장하세요


In [ ]:
np.random.seed(123) # 123번 시드로 난수 세트 고정

sample = np.random.choice(e_pub, size = 10) # 무작위로 10개 구 가져와서 123번 시드에 저장

print(np.mean(sample)) # 10개 구 이팝나무 그루 수 평균

print(stats.ttest_1samp(sample, 400)) # T-test(1 sample) 결과

pv2 = stats.ttest_1samp(sample, 400).pvalue # P-value 값 저장
pv2
# pv2 > 0.05 이므로 H0 는 채택(서울시에는 구별로 평균 약 400 그루 정도의 이팝나무가 있을 수도 있다)

479.7
Ttest_1sampResult(statistic=0.706555193453698, pvalue=0.49772576203804575)


0.49772576203804575

## 2. Two-Sample T-test 연습

### "서울시의 구별 평균 느티나무수와 왕벚나무의 수는 차이가 없다"라는 가설에 대해 가설검정을 시행하세요.

- 단, `서울시`, `공원녹지사업소`, `시설관리공단` 부분의 데이터는 제외합니다.
- 가설검정 결과의 pvalue를 `pv3`에 저장하세요

In [ ]:
# H0 = 서울시의 구별 평균 느티나무수와 왕벚나무의 수는 차이가 없다
# H1 = 서울시의 구별 평균 느티나무수와 왕벚나무의 수는 차이가 있다
n_ti = df['느티나무']
king_sakura = df['왕벚나무']
print(np.mean(n_ti))
print(np.mean(king_sakura))

print(stats.ttest_ind(n_ti, king_sakura))

pv3 = stats.ttest_ind(n_ti, king_sakura).pvalue
pv3
# pv3 > 0.05 이므로 H0는 채택 (서울시의 구별 평균 느티나무수와 왕벚나무의 수는 차이가 없을 수도 있다)

1415.12
1292.0
Ttest_indResult(statistic=0.4629256014492562, pvalue=0.6455096880085703)


0.6455096880085703


### Two-Sample One-tailed test를 사용하여 **평균 느티나무수가 왕벚나무 수보다 크다**라는 가설에 대한 가설검정을 시행하세요.
- 해당 가설검정의 결과를 `ans`에 저장하세요.

이 때 `ans`가 가질 수 있는 값은 아래와 같습니다 :
1. "느티나무" ( 느티나무의 수가 통계적으로 유의하게 많은 경우 )
2. "왕벚나무" ( 왕벚나무의 수가 통계적으로 유의하게 많은 경우 )
3. "없음" (통계적으로 유의하지 않은 경우)

Two-Sample One-tailed test를 scipy에서 하는 방법은 아래의 링크를 참고 하세요  
참고 링크 : https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_ind.html

In [ ]:
# 아래의 코드를 실행해주세요
!pip install scipy --upgrade

In [ ]:
# 평균 느티나무수가 왕벚나무 수보다 크다
# H0 : np.mean(n_ti) > np.mean(king_sakura)
# H1 : np.mean(n_ti) <= np.mean(king_sakura)

t_ans, pv_ans = stats.ttest_ind(n_ti, king_sakura, alternative='less')
print(t_ans)
print(pv_ans/ 2) # One-tailed test 이므로 2로 나눔

if((pv_ans / 2) > 0.05):
  ans = '없음'
elif((t_ans > 0) & ((pv_ans / 2) <= 0.05)):
  ans = '느티나무'
else:
  ans = '왕벚나무'

ans

0.4629256014492562
0.33862257799785744


'없음'

## 🔥 **도전과제**
> 2문제 모두 하면 됩니다.



### 1. Compare mean for multiple-sample 
 
다음 링크를 참조하세요. <https://reneshbedre.github.io/blog/anova.html>

구별, 은행나무, 양버즘나무, 느티나무에 대해서 ANOVA Test를 실행하세요.

단, `서울시`, `공원녹지사업소`, `시설관리공단` 부분의 데이터는 제외합니다.

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
import warnings
warnings.filterwarnings('ignore')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
# Null hypothesis: Groups means are equal (no variation in means of groups) 
# H0: μ1=μ2=…=μp
# Alternative hypothesis: At least, one group mean is different from other groups
# H1: All μ are not equal

stats.f_oneway(df['은행나무'], df['양버즘나무'], df['느티나무'])

F_onewayResult(statistic=17.006289557888046, pvalue=8.935183167883698e-07)

### 2. Sampling

- 세션 시간에 배웠던 4가지 sampling (Simple Random, Systematic, Stratified Random, Cluster) 방식을 구현해보세요.

- 이를 위한 데이터는 1 부터 100을 담은 numpy array이며, Sampling 해야 하는 수는 20개 입니다.

- Stratified Random Sampling을 위한 그룹을 나누는 기준은 1-10, 11-20, ... 과 같은 방식입니다.

- Cluster Sampling에서 그룹을 나누는 기준은 5로 나눴을때의 나머지 값입니다. (0,1,2,3,4)

In [ ]:
# 1 부터 100을 담은 numpy array
array = np.arange(1, 101)
print(array)

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100]


In [ ]:
# Simple Random
simp_rd = np.random.choice(array, size = 20)
print(simp_rd)

[48 74 33 47 97 26 84 79 37 97 81 69 50 56 68  3 85 40 67 85]


In [ ]:
# Systematic
# 참고 : https://www.geeksforgeeks.org/systematic-sampling-in-pandas/
def systematic_sampling(array, step):
    indexes = np.arange(0, len(array), step=step)
    systematic_sample = array[indexes]
    return systematic_sample

sys = systematic_sampling(array, 5)
print(sys)

[ 1  6 11 16 21 26 31 36 41 46 51 56 61 66 71 76 81 86 91 96]


In [ ]:
# Stratified Random
# 10개씩 분할하기
array_grouped = []
for idx in range(0, 10):
  add = 10
  start = add*idx # 0, 10, 20, ..., 90
  end = add*idx + 9 # 9, 19, 29, ..., 99
  array_grouped.append(array[start:end])

# 각 그룹에서 2개씩 무작위 추출하기
str_rd = []
for i in range(10):
  str_rd.extend(np.random.choice(array_grouped[i], size = 2))

print(str_rd)

[9, 1, 18, 14, 25, 27, 32, 36, 47, 43, 52, 59, 64, 66, 71, 73, 87, 83, 95, 95]


In [18]:
# Cluster
array_cluster_0 = array[(array % 5 == 0)]
array_cluster_1 = array[(array % 5 == 1)]
array_cluster_2 = array[(array % 5 == 2)]
array_cluster_3 = array[(array % 5 == 3)]
array_cluster_4 = array[(array % 5 == 4)]
array_cluster = [array_cluster_0, array_cluster_1, array_cluster_2, array_cluster_3, array_cluster_4]

cluster = []
for i in range(5):
  cluster.extend(np.random.choice(array_cluster[i], size = 4))

print(cluster)

[35, 45, 55, 65, 61, 51, 86, 36, 67, 62, 7, 47, 73, 8, 18, 88, 74, 29, 9, 94]
